[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/romiaprilian7406/sp500-risk-return-clustering/blob/main/notebooks/sp500_risk_return_dataset.ipynb)


# Import Library

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm.auto import tqdm
import time
import warnings

warnings.filterwarnings('ignore')

# Global Configuration

In [2]:
DATA_URL = "https://raw.githubusercontent.com/romiaprilian7406/sp500-companies/main/data/sp500_companies.csv"
EXCLUDED_SECTORS = ['Financials', 'Real Estate'] # Sektor dengan struktur neraca berbeda
REQUEST_DELAY = 0.5

# Helper Functions

In [3]:
def load_and_filter_tickers(url, excluded_sectors):
    df = pd.read_csv(url)
    df_filtered = df[~df['GICS Sector'].isin(excluded_sectors)].copy()
    df_filtered['Symbol'] = df_filtered['Symbol'].str.replace('.', '-', regex=False)
    return df_filtered

def safe_get(source, keys):
    if isinstance(source, dict):
        val = source.get(keys, np.nan)
        return float(val) if val is not None else np.nan
    if isinstance(keys, str): keys = [keys]
    for k in keys:
        if k in source:
            val = source[k]
            return float(val) if not pd.isna(val) else np.nan
    return np.nan

# Mengambil Data Fundamental + Data Historis Harga
def extract_data(ticker):
    try:
        stock = yf.Ticker(ticker)

        # A. Fetch Fundamental Info
        info = stock.info

        # B. Fetch Financial Statements
        try:
            inc = stock.financials.iloc[:, 0] if not stock.financials.empty else pd.Series(dtype=float)
            bal = stock.balance_sheet.iloc[:, 0] if not stock.balance_sheet.empty else pd.Series(dtype=float)
        except:
            inc, bal = pd.Series(dtype=float), pd.Series(dtype=float)

        # C. Fetch Historical Prices (PENTING untuk Clustering Risiko)
        # Ambil 3 tahun terakhir untuk menghitung Volatilitas & CAGR
        try:
            hist = stock.history(period="3y")
        except:
            hist = pd.DataFrame()

        # D. Construct Raw Dictionary
        raw = {
            # Valuation & Size
            'EnterpriseValue': safe_get(info, 'enterpriseValue'),
            'EBITDA': safe_get(inc, ['EBITDA', 'Normalized EBITDA']),
            'TotalRevenue': safe_get(inc, 'Total Revenue'),
            'MarketCap': safe_get(info, 'marketCap'),

            # Profitability
            'EBIT': safe_get(inc, ['EBIT', 'Operating Income']),
            'PretaxIncome': safe_get(inc, ['Pretax Income', 'Income Before Tax']),
            'TaxProvision': safe_get(inc, ['Tax Provision', 'Income Tax Expense']),
            'TotalAssets': safe_get(bal, 'Total Assets'),
            'TotalEquity': safe_get(bal, ['Stockholders Equity', 'Total Equity Gross Minority Interest']),
            'TotalDebt': safe_get(bal, ['Total Debt', 'Total Liab']),
            'CashAndEquivalents': safe_get(bal, ['Cash And Cash Equivalents', 'Cash']),

            # Dividends
            'DividendYield': safe_get(info, 'dividendYield'), # Penting untuk cluster "Income Stock"

            # Historical Data Container
            'History': hist
        }
        return raw

    except Exception as e:
        print(f"Error extracting {ticker}: {e}")
        return None

def transform_data(raw, ticker, sector):
    try:
        # 1. Viability Check
        if pd.isna(raw['EBITDA']) or pd.isna(raw['MarketCap']) or raw['History'].empty:
            return None

        # 2. FEATURE ENGINEERING
        # A. ROIC (Quality Metric)
        equity = raw['TotalEquity'] if not pd.isna(raw['TotalEquity']) else (raw['TotalAssets'] - raw['TotalDebt'])
        invested_capital = equity + raw['TotalDebt'] - raw['CashAndEquivalents']

        if invested_capital <= 0: return None

        tax_rate = 0.21
        if raw['PretaxIncome'] != 0 and not pd.isna(raw['PretaxIncome']):
            tax_rate = raw['TaxProvision'] / raw['PretaxIncome']
            # Cap tax rate agar masuk akal (0% - 40%)
            tax_rate = max(0.0, min(tax_rate, 0.40))

        nopat = raw['EBIT'] * (1 - tax_rate)
        roic = nopat / invested_capital

        # B. Volatility (Risk Metric)
        hist = raw['History']
        if len(hist) < 250: return None

        daily_returns = hist['Close'].pct_change().dropna()
        annualized_volatility = daily_returns.std() * np.sqrt(252) # 252 hari bursa

        # C. Momentum / CAGR (Growth Metric)
        start_price = hist['Close'].iloc[0]
        end_price = hist['Close'].iloc[-1]
        years = len(hist) / 252
        price_cagr = (end_price / start_price) ** (1/years) - 1

        # D. Dividend Yield (Income Metric)
        div_yield = raw['DividendYield'] if not pd.isna(raw['DividendYield']) else 0.0

        # E. Size
        revenue = raw['TotalRevenue']

        return {
            'Ticker': ticker,
            'Sector': sector,
            'ROIC': roic,
            'Volatility': annualized_volatility,
            'Dividend_Yield': div_yield,
            'Price_CAGR': price_cagr,
            'TotalRevenue': revenue,
            'EV_EBITDA': raw['EnterpriseValue'] / raw['EBITDA']
        }

    except Exception as e:
        return None

# Execution

In [4]:
df_sp500 = load_and_filter_tickers(DATA_URL, EXCLUDED_SECTORS)
tickers = df_sp500['Symbol'].tolist()
sector_map = dict(zip(df_sp500['Symbol'], df_sp500['GICS Sector']))

results = []
print(f"Memulai pengambilan data untuk {len(tickers)} saham")

for ticker in tqdm(tickers, desc="Fetching Data"):
    sector = sector_map.get(ticker, 'Unknown')

    # 1. EXTRACT (Ambil Data Mentah)
    raw_data = extract_data(ticker)

    # 2. TRANSFORM (Validasi & Hitung Rumus)
    if raw_data:
        clean_data = transform_data(raw_data, ticker, sector)

        # Jika lolos semua filter di transform, simpan
        if clean_data:
            results.append(clean_data)

    time.sleep(REQUEST_DELAY)

print(f"Total saham yang berhasil diambil: {len(results)} ")

Memulai pengambilan data untuk 396 saham


Fetching Data:   0%|          | 0/396 [00:00<?, ?it/s]

Total saham yang berhasil diambil: 384 


In [5]:
if results:
    df_final = pd.DataFrame(results)

    # Tampilkan preview
    print("Preview data:")
    display(df_final.head())
else:
    print("GAGAL: Tidak ada data")

Preview data:


,Ticker,Sector,ROIC,Volatility,Dividend_Yield,Price_CAGR,TotalRevenue,EV_EBITDA
0,MMM,Industrials,0.420746,0.303175,1.81,0.210768,2.457500e+10,12.745011
1,AOS,Industrials,0.289530,0.255343,2.02,0.087453,3.818100e+09,12.135190
2,ABT,Health Care,0.137500,0.200555,2.01,0.075907,4.195000e+10,20.678374
3,ABBV,Health Care,0.100454,0.232020,3.11,0.155468,5.633400e+10,30.652559
4,ACN,Information Technology,0.286983,0.252126,2.42,0.034539,6.967298e+10,13.963515


# Simple EDA

In [6]:
# Shape
rows, cols = df_final.shape
print(f"Dimensi Dataset")
print(f"Total Baris    : {rows}")
print(f"Total Kolom    : {cols}\n")

# Info
print(df_final.info(),"\n")

# Fitur adalah semua kolom selain Target dan Metadata
features_list = [col for col in df_final.columns]

print(f"Jumlah Fitur : {len(features_list)}")
print(f"Daftar Fitur : {features_list}\n")

# Statistik Deskriptif Fitur Numerik
print(f"\nStatistik Deskriptif")
numeric_cols = df_final.select_dtypes(include=[np.number]).columns.tolist()

try:
    display(df_final[numeric_cols].describe().T.round(2))
except:
    print(df_final[numeric_cols].describe().T.round(2))

# Missing Values
print(f"\nLaporan Missing Values:")
missing_count = df_final.isnull().sum()
missing_pct = (df_final.isnull().sum() / len(df_final)) * 100

missing_df_final = pd.DataFrame({
    'Jumlah Hilang': missing_count,
    'Persentase (%)': missing_pct
})

# Filter hanya yang ada missing value & Sort dari yang terbanyak
missing_only = missing_df_final[missing_df_final['Jumlah Hilang'] > 0].sort_values(by='Jumlah Hilang', ascending=False)

# Format persentase setelah sorting
missing_only['Persentase (%)'] = missing_only['Persentase (%)'].map('{:.2f}%'.format)

if not missing_only.empty:
    print(missing_only)
else:
    print("Tidak ada missing values")

Dimensi Dataset
Total Baris    : 384
Total Kolom    : 8

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ticker          384 non-null    object 
 1   Sector          384 non-null    object 
 2   ROIC            382 non-null    float64
 3   Volatility      384 non-null    float64
 4   Dividend_Yield  384 non-null    float64
 5   Price_CAGR      384 non-null    float64
 6   TotalRevenue    384 non-null    float64
 7   EV_EBITDA       381 non-null    float64
dtypes: float64(6), object(2)
memory usage: 24.1+ KB
None 

Jumlah Fitur : 8
Daftar Fitur : ['Ticker', 'Sector', 'ROIC', 'Volatility', 'Dividend_Yield', 'Price_CAGR', 'TotalRevenue', 'EV_EBITDA']


Statistik Deskriptif


,count,mean,std,min,25%,50%,75%,max
ROIC,382.0,1.700000e-01,1.800000e-01,-9.000000e-01,7.000000e-02,1.300000e-01,2.200000e-01,1.090000e+00
Volatility,384.0,3.000000e-01,1.000000e-01,1.500000e-01,2.300000e-01,2.900000e-01,3.500000e-01,9.500000e-01
Dividend_Yield,384.0,1.740000e+00,1.720000e+00,0.000000e+00,2.800000e-01,1.300000e+00,2.780000e+00,1.273000e+01
Price_CAGR,384.0,1.600000e-01,2.900000e-01,-4.600000e-01,1.000000e-02,1.100000e-01,2.300000e-01,3.210000e+00
TotalRevenue,384.0,3.659847e+10,7.447041e+10,7.058230e+08,7.098575e+09,1.465271e+10,3.018450e+10,6.809850e+11
EV_EBITDA,381.0,2.438000e+01,7.025000e+01,-1.195800e+02,1.134000e+01,1.541000e+01,2.151000e+01,1.276280e+03



Laporan Missing Values:
           Jumlah Hilang Persentase (%)
EV_EBITDA              3          0.78%
ROIC                   2          0.52%


# Export Dataset

In [8]:
file_name = 'sp500_risk_return_dataset.csv'
df_final.to_csv(file_name, index=False)
print(f"File dataset disimpan ke: {file_name}")

File dataset disimpan ke: sp500_risk_return_dataset.csv
